In [2]:
# https://keras.io/
!pip install -q keras
import keras
import cv2

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD, Adadelta
from keras import optimizers
import numpy as np


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 27s 0us/step
32 32 3


In [8]:
def resize_data(x):
    x_scaled = np.zeros((x.shape[0], 24, 24, 3))
    for i, img in enumerate(x):
        new_image = cv2.resize(img, dsize=(24, 24), interpolation=cv2.INTER_CUBIC)
        x_scaled[i] = new_image

    return x_scaled

# resize train and  test data
x_train_24 = resize_data(x_train)
x_test_24 = resize_data(x_test)

img_height_24, img_width_24, channel = x_train_24.shape[1],x_train_24.shape[2],x_train_24.shape[3]
print(img_height_24, img_width_24, channel)

24 24 3


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 96, dropout_rate = 0.25):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 96, dropout_rate = 0.25):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 96
dropout_rate = 0.25
l = 12

input_small = Input((img_height_24, img_width_24, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input_small)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output_small = output_layer(Last_Block)


In [13]:
model = Model(inputs=[input_small], outputs=[output_small])
model.summary()
#model = Model(inputs=[input], outputs=[output])
#model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 24, 96)   2592        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 24, 24, 96)   384         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 24, 24, 96)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
from keras.callbacks import ModelCheckpoint
filepath="weights.best_cifar10_small.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer= Adam(amsgrad=True),
              metrics=['accuracy'])

In [0]:
model.fit(x_train_24, y_train,
                    batch_size=batch_size,
                    epochs=15,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test_24, y_test))

In [74]:
# Test the model
score = model.evaluate(x_test_24, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 41s 4ms/step
Test loss: 0.5849459414362908
Test accuracy: 0.8447


In [75]:
#removing the last two layers so that we can modify the model to accomodate original images
model.layers.pop()
model.layers.pop()

In [0]:
# Save the trained weights in to .h5 format
model.save_weights("cifar_10_model_24_small.h5")
from google.colab import files
files.download("cifar_10_model_24_small.h5")
files.download("weights.best_cifar10_small.hdf5")
print("Saved model to disk")

In [21]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

Saving cifar_10_model_24_small.h5 to cifar_10_model_24_small.h5
User uploaded file "cifar_10_model_24_small.h5" with length 28029240 bytes


In [0]:
num_filter = 96
dropout_rate = 0.25
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
BatchNorm = BatchNormalization()(Last_Block)
relu = Activation('relu')(BatchNorm)
output_big = AveragePooling2D(pool_size=(2,2))(relu)


In [18]:
model_big = Model(inputs=[input], outputs=[output_big])
model_big.summary()

# determine Loss function and Optimizer
model_big.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 96)   2592        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 96)   384         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 96)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
model_big.load_weights('cifar_10_model_24_small.h5')

In [0]:
flat = Flatten()(model_big.output)
output = Dense(num_classes, activation='softmax')(flat)

In [0]:
model_big = Model(inputs=[model_big.input], outputs=[output])

In [25]:
model_big.compile(loss='categorical_crossentropy',
              optimizer=Adam(amsgrad = True),
             metrics=['accuracy'])
model_big.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_53 (Conv2D)              (None, 32, 32, 96)   2592        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_53 (BatchNo (None, 32, 32, 96)   384         conv2d_53[0][0]                  
__________________________________________________________________________________________________
activation_53 (Activation)      (None, 32, 32, 96)   0           batch_normalization_53[0][0]     
__________________________________________________________________________________________________
conv2d_54 

In [0]:
from keras.callbacks import ModelCheckpoint
filepath="weights.best_cifar10_big.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [26]:
# image augumentation 
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot 

datagen = ImageDataGenerator(
        rotation_range=40,
        vertical_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

datagen.fit(x_train)

model_big.fit_generator(datagen.flow(x_train, y_train, batch_size=64), samples_per_epoch=len(x_train), epochs=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, steps_per_epoch=781)`
  app.launch_new_instance()


Epoch 1/5
781/781 [==============================] - 763s 977ms/step - loss: 1.2324 - acc: 0.5654
Epoch 2/5
150/781 [====>.........................] - ETA: 9:58 - loss: 1.0964 - acc: 0.6181

781/781 [==============================] - 743s 951ms/step - loss: 1.0381 - acc: 0.6366
Epoch 3/5
271/781 [=========>....................] - ETA: 8:04 - loss: 0.9692 - acc: 0.6612

781/781 [==============================] - 743s 952ms/step - loss: 0.9436 - acc: 0.6700
Epoch 4/5
326/781 [===========>..................] - ETA: 7:13 - loss: 0.9017 - acc: 0.6872

781/781 [==============================] - 743s 952ms/step - loss: 0.8819 - acc: 0.6927
Epoch 5/5
351/781 [============>.................] - ETA: 6:48 - loss: 0.8429 - acc: 0.7064

781/781 [==============================] - 744s 952ms/step - loss: 0.8218 - acc: 0.7127


In [27]:
model_big.fit(x_train, y_train,
                    batch_size=112,
                    epochs=10,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 753s 15ms/step - loss: 0.3507 - acc: 0.8790 - val_loss: 0.5133 - val_acc: 0.8529

Epoch 00001: val_acc improved from -inf to 0.85290, saving model to weights.best_cifar10_small.hdf5
Epoch 2/10
 9744/50000 [====>.........................] - ETA: 9:10 - loss: 0.2640 - acc: 0.9071

50000/50000 [==============================] - 736s 15ms/step - loss: 0.2635 - acc: 0.9079 - val_loss: 0.4967 - val_acc: 0.8583

Epoch 00002: val_acc improved from 0.85290 to 0.85830, saving model to weights.best_cifar10_small.hdf5
Epoch 3/10
23408/50000 [=============>................] - ETA: 6:03 - loss: 0.2213 - acc: 0.9252

50000/50000 [==============================] - 736s 15ms/step - loss: 0.2221 - acc: 0.9236 - val_loss: 0.4920 - val_acc: 0.8616

Epoch 00003: val_acc improved from 0.85830 to 0.86160, saving model to weights.best_cifar10_small.hdf5
Epoch 4/10
26992/50000 [===============>..............] - ETA: 5:14 - loss: 0.1946 - acc: 0.9306

50000/50000 [==============================] - 737s 15ms/step - loss: 0.1973 - acc: 0.9296 - val_loss: 0.3648 - val_acc: 0.8961

Epoch 00004: val_acc improved from 0.86160 to 0.89610, saving model to weights.best_cifar10_small.hdf5
Epoch 5/10
27888/50000 [===============>..............] - ETA: 5:02 - loss: 0.1674 - acc: 0.9422

50000/50000 [==============================] - 738s 15ms/step - loss: 0.1703 - acc: 0.9409 - val_loss: 0.5326 - val_acc: 0.8626

Epoch 00005: val_acc did not improve from 0.89610
Epoch 6/10
33712/50000 [===================>..........] - ETA: 3:42 - loss: 0.1539 - acc: 0.9454

50000/50000 [==============================] - 736s 15ms/step - loss: 0.1548 - acc: 0.9448 - val_loss: 0.5106 - val_acc: 0.8721

Epoch 00006: val_acc did not improve from 0.89610
Epoch 7/10
35504/50000 [====================>.........] - ETA: 3:18 - loss: 0.1375 - acc: 0.9527

50000/50000 [==============================] - 737s 15ms/step - loss: 0.1400 - acc: 0.9515 - val_loss: 0.7261 - val_acc: 0.8439

Epoch 00007: val_acc did not improve from 0.89610
Epoch 8/10
36064/50000 [====================>.........] - ETA: 3:10 - loss: 0.1238 - acc: 0.9561

50000/50000 [==============================] - 737s 15ms/step - loss: 0.1259 - acc: 0.9555 - val_loss: 0.4674 - val_acc: 0.8798

Epoch 00008: val_acc did not improve from 0.89610
Epoch 9/10
36176/50000 [====================>.........] - ETA: 3:09 - loss: 0.1143 - acc: 0.9588

50000/50000 [==============================] - 739s 15ms/step - loss: 0.1134 - acc: 0.9596 - val_loss: 0.4149 - val_acc: 0.8957

Epoch 00009: val_acc did not improve from 0.89610
Epoch 10/10
36064/50000 [====================>.........] - ETA: 3:11 - loss: 0.0980 - acc: 0.9656

50000/50000 [==============================] - 740s 15ms/step - loss: 0.1010 - acc: 0.9645 - val_loss: 0.6883 - val_acc: 0.8533

Epoch 00010: val_acc did not improve from 0.89610


In [0]:
model_big.save_weights("weights.best_cifar10_big.hdf5")
print("Saved model to disk")
from google.colab import files
files.download("weights.best_cifar10_big.hdf5")
print("Saved model to disk")

In [0]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [87]:
#image augumentation 
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot 

datagen = ImageDataGenerator(
        rotation_range=40,
        vertical_flip = True,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

datagen.fit(x_train)

model_big.fit_generator(datagen.flow(x_train, y_train, batch_size=64), samples_per_epoch=len(x_train), epochs=5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, steps_per_epoch=781)`
  app.launch_new_instance()


Epoch 1/5
781/781 [==============================] - 741s 949ms/step - loss: 0.9680 - acc: 0.6677
Epoch 2/5
151/781 [====>.........................] - ETA: 9:58 - loss: 0.8439 - acc: 0.7035

781/781 [==============================] - 741s 949ms/step - loss: 0.8140 - acc: 0.7133
Epoch 3/5
272/781 [=========>....................] - ETA: 8:02 - loss: 0.7697 - acc: 0.7313

781/781 [==============================] - 741s 948ms/step - loss: 0.7600 - acc: 0.7346
Epoch 4/5
326/781 [===========>..................] - ETA: 7:11 - loss: 0.7362 - acc: 0.7418

781/781 [==============================] - 741s 949ms/step - loss: 0.7266 - acc: 0.7472
Epoch 5/5
351/781 [============>.................] - ETA: 6:48 - loss: 0.6796 - acc: 0.7606

781/781 [==============================] - 742s 950ms/step - loss: 0.6900 - acc: 0.7586


In [0]:
model_big.fit(x_train, y_train,
                    batch_size=112,
                    epochs=10,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

In [0]:
model_big.save_weights("weights.best_cifar10_big.hdf5")
print("Saved model to disk")